<a href="https://colab.research.google.com/github/ckjen168/LLMColab/blob/main/%E5%8F%B0%E7%81%A3%E4%BA%A4%E9%80%9A%E6%B3%95%E8%A6%8F%E5%95%8F%E7%AD%94%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第 7 章 用 RAG 讓模型增加額外知識

In [1]:
!pip install langchain langchain_openai langchain_community rich --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 981.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.1 MB/s eta 0:00:00


In [2]:
# 匯入套件和金鑰
from google.colab import userdata
from rich import print as pprint
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI()

## 7-1 什麼是 RAG?

### RAG 第一步：資料匯入

In [3]:
!pip install pypdf chromadb rapidocr-onnxruntime --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.2 MB/s eta 0:00

extract_images 設定 True 可以解析帶有圖片的 PDF

In [4]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_path='https://ppt.cc/f9nc5x')

In [5]:
docs = loader.load()
pprint(docs[0])

Document(
    metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 0},
    page_content='汽車法規是非題  \n第1頁/共 36頁  \n                      分類編號欄位說明  \n分類編\n號  
分類項目內容  \n01  路口安全(有號誌路口、無號誌路口、停讓行人)  \n02  轉彎(左右轉、迴轉) \n03  
行駛中應注意事項(保持安全車距、注意前車狀況) \n04  正確使用燈光(頭燈、霧燈、方向燈) \n05  
注意大型車行駛及轉彎(內輪差、視野死角、不並行) \n06  貨物裝載(防止掉落或滲漏) \n07 
事故預防及處理(預防國道二次事故、急救常識) \n08 禁止不當行為(酒駕、不使用手機、危險駕駛) \n09 行車檢查(設備、燈光) 
\n10 其他(平交道、強制險、環保駕駛、特殊天候、駕駛道德) \n  '
)

In [6]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
embeddings_model=OpenAIEmbeddings(model='text-embedding-3-large')
index = VectorstoreIndexCreator(
    embedding=embeddings_model).from_loaders([loader])

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [7]:
query = "酒後開車且酒精濃度超過規定標準應罰款多少?"
response = index.query(llm=chat_model, question=query)
print(response)

酒後開車且酒精濃度超過規定標準應處以新臺幣30,000到120,000元罰款，並當場移置保管汽車及吊銷駕照。


### RAG 第二步：資料分割

In [8]:
test_doc = docs[1].page_content[:200]
test_doc

'汽車法規是非題  \n第2頁/共 36頁  \n  \n題號 答案 題   目  \n分類\n編號  \n001  ○  \n尊重生命是駕駛道德最重要的一點，我們開車時要處處顧及行人，尤\n其應該注意讓老弱婦孺身心障礙者優先通行。  10 \n002  X  遵守交通法規與秩序，只算是優良駕駛人，與駕駛道德無關。  10 \n003  ○  汽油著火時，應用滅火器、泥沙或用水浸濕棉被、衣服覆蓋撲滅。  07 \n004'

In [14]:
from langchain_text_splitters import (
    CharacterTextSplitter, RecursiveCharacterTextSplitter)

In [15]:
text_splitter = CharacterTextSplitter(separator='',chunk_size=10,chunk_overlap=2)
chunks = text_splitter.split_text(test_doc)
pprint(chunks)

[
    '汽車法規是非題',
    '第2頁/共 36',
    '36頁  \n  \n題',
    '題號 答案 題',
    '目  \n分類',
    '類\n編號  \n001',
    '01  ○  \n尊重',
    '尊重生命是駕駛道德最',
    '德最重要的一點，我們',
    '我們開車時要處處顧及',
    '顧及行人，尤\n其應該',
    '應該注意讓老弱婦孺身',
    '孺身心障礙者優先通行',
    '通行。  10 \n0',
    '002  X  遵',
    '遵守交通法規與秩序',
    '秩序，只算是優良駕駛',
    '駕駛人，與駕駛道德無',
    '德無關。  10',
    '003  ○',
    '汽油著火時，應用',
    '應用滅火器、泥沙或用',
    '或用水浸濕棉被、衣服',
    '衣服覆蓋撲滅。  0',
    '07 \n004'
]

In [16]:
text_splitter = RecursiveCharacterTextSplitter(separators=['○','X'],chunk_size=10,chunk_overlap=3)
chunks = text_splitter.split_text(test_doc)
pprint(chunks)

[
    '汽車法規是非題  \n第2頁/共 36頁  \n  \n題號 答案 題   目  \n分類\n編號  \n001  ',
    '○  \n尊重生命是駕駛道德最重要的一點，我們開車時要處處顧及行人，尤\n其應該注意讓老弱婦孺身心障礙者優先通行。  
10 \n002  ',
    'X  遵守交通法規與秩序，只算是優良駕駛人，與駕駛道德無關。  10 \n003  ',
    '○  汽油著火時，應用滅火器、泥沙或用水浸濕棉被、衣服覆蓋撲滅。  07 \n004'
]

In [17]:
from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=10,
                                  chunk_overlap=2)
chunks = text_splitter.split_text(test_doc)
pprint(chunks)

[
    '汽車法規是',
    '�是非題  ',
    '  \n第2頁',
    '��/共 36頁 ',
    '�  \n  \n題',
    '��號 答案',
    '�� 題   目',
    '��  \n分類',
    '��\n編號  ',
    '  \n001  ○  \n',
    ' \n尊重生命是',
    '�是駕駛道',
    '道德最重要',
    '要的一點，',
    '��我們開�',
    '�車時要處',
    '��處顧及',
    '及行人，尤',
    '尤\n其應該',
    '��注意讓�',
    '�老弱婦孺',
    '孺身心障�',
    '�礙者優先',
    '先通行。  10 ',
    ' 10 \n002  X  遵',
    '��守交通法',
    '法規與秩�',
    '�序，只�',
    '算是優良',
    '��駕駛人�',
    '人，與駕�',
    '�駛道德無',
    '無關。  10 \n003',
    '\n003  ○  汽�',
    '�油著火�',
    '��時，應�',
    '�用滅火�',
    '器、泥沙�',
    '�或用水浸�',
    '�濕棉被',
    '��、衣服覆�',
    '�蓋撲滅',
    '��。  07 \n004'
]

In [18]:
from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(model_name='gpt-4-turbo',
                                  chunk_size=10,
                                  chunk_overlap=2)
chunks = text_splitter.split_text(test_doc)
pprint(chunks)

[
    '汽車法規是非�',
    '非題  \n第2頁/共',
    '/共 36頁  \n  \n題�',
    '�號 答案 題',
    '題   目  \n分類\n',
    '�\n編號  \n001  �',
    '  ○  \n尊重生命是',
    '命是駕駛道�',
    '道德最重要的一點',
    '點，我們開車',
    '��時要處處顧',
    '顧及行人，尤\n其',
    '\n其應該注意讓',
    '讓老弱婦孺',
    '孺身心障礙者�',
    '者優先通行。  ',
    '  10 \n002  X  �',
    ' 遵守交通法規',
    '規與秩序，只算',
    '只算是優良�',
    '駕駛人，與',
    '與駕駛道�',
    '道德無關。  10',
    ' 10 \n003  ○  �',
    ' 汽油著火時',
    '火時，應用滅火',
    '�火器、泥沙或用',
    '或用水浸濕棉',
    '棉被、衣服覆',
    '覆蓋撲滅。',
    '�。  07 \n004'
]

In [19]:
text_splitter = RecursiveCharacterTextSplitter(separators=[' \n'],
                                               chunk_size=10,
                                               chunk_overlap=2)
chunks = text_splitter.split_documents(docs)
pprint(chunks[15:20])

[
    Document(metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}, page_content=' \n第2頁/共 36頁 '),
    Document(metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}, page_content=' \n題號 答案 題   目 '),
    Document(metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}, page_content='分類\n編號'),
    Document(metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1}, page_content='001  ○'),
    Document(
        metadata={'source': 'https://ppt.cc/f9nc5x', 'page': 1},
        page_content=' 
\n尊重生命是駕駛道德最重要的一點，我們開車時要處處顧及行人，尤\n其應該注意讓老弱婦孺身心障礙者優先通行。  10'
    )
]

## 7-2 Embedding 向量化

### RAG 第三步：文字轉向量

In [20]:
embeddings_doc = [
    "天空是藍色的",
    "天空不是紅色的",
    "sky is blue",
    "莓果是藍色的",
    "我今天吃了漢堡"
]
embeddings = embeddings_model.embed_documents(embeddings_doc)
len(embeddings[0])

3072

In [21]:
query = "天空的顏色是？"
embedded_query = embeddings_model.embed_query(query)

餘弦相似度

In [22]:
import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b)

In [23]:
for doc_res, doc in zip(embeddings, embeddings_doc):
    similarity = cosine_similarity(embedded_query,doc_res)
    print(f'"{doc}" 與問題的相似度：{similarity}')

"天空是藍色的" 與問題的相似度：0.750690576104704
"天空不是紅色的" 與問題的相似度：0.6954274413696448
"sky is blue" 與問題的相似度：0.5886609903775393
"莓果是藍色的" 與問題的相似度：0.3678271743679623
"我今天吃了漢堡" 與問題的相似度：0.08472869956688046


### RAG 第四步：儲存到向量資料庫 Chroma

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
from langchain_community.vectorstores import Chroma
Chroma.from_documents(documents=chunks,
                embedding=embeddings_model,
                persist_directory='/content/drive/MyDrive/db',
                collection_metadata={"hnsw:space": "cosine"})

hnsw:space 有 l2、ip、cosine

In [26]:
db = Chroma(persist_directory='/content/drive/MyDrive/db',
            embedding_function=embeddings_model)

<ipython-input-26-7ab32535ab52>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory='/content/drive/MyDrive/db',


In [27]:
pprint(db.search('紅燈右轉', k=2, search_type='similarity'))

[
    Document(
        metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'},
        page_content=' \n汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 30 
公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。 '
    ),
    Document(
        metadata={'page': 24, 'source': 'https://ppt.cc/f9nc5x'},
        page_content=' \n447  X  在交岔路口等候左轉之車輛，綠燈亮時即可搶先左轉。  02'
    )
]

In [28]:
pprint(db.max_marginal_relevance_search("紅燈右轉",k=2))

[
    Document(
        metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'},
        page_content=' \n汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 30 
公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。 '
    ),
    Document(
        metadata={'page': 1, 'source': 'https://ppt.cc/f9nc5x'},
        page_content=' \n006  ○  綠燈允許你依序通過，但駕駛人仍應注意違規闖紅燈的人和車。  01'
    )
]

In [29]:
pprint(db.similarity_search_with_relevance_scores('紅燈右轉',k=2))

[
    (
        Document(
            metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'},
            page_content=' \n汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 30 
公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。 '
        ),
        0.6080599427223206
    ),
    (
        Document(
            metadata={'page': 24, 'source': 'https://ppt.cc/f9nc5x'},
            page_content=' \n447  X  在交岔路口等候左轉之車輛，綠燈亮時即可搶先左轉。  02'
        ),
        0.5934140682220459
    )
]

In [30]:
embedded_query = embeddings_model.embed_query("紅燈右轉")
pprint(db.similarity_search_by_vector(embedded_query ,k=2))

[
    Document(
        metadata={'page': 12, 'source': 'https://ppt.cc/f9nc5x'},
        page_content=' \n汽車行駛至交岔路口，欲右轉彎時，應距交岔路口 30 
公尺前，顯示\n方向燈或手勢，換入外側車道、右轉車道或慢車道，行至路口後再行\n右轉。 '
    ),
    Document(
        metadata={'page': 24, 'source': 'https://ppt.cc/f9nc5x'},
        page_content=' \n447  X  在交岔路口等候左轉之車輛，綠燈亮時即可搶先左轉。  02'
    )
]

## 7-3 檢索對話流程鏈

檢索器的search_type可以設定 similarity、mmr、similarity_score_threshold

In [31]:
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={"k": 6})

In [32]:
retrieved_docs = retriever.invoke("紅燈右轉")
print(f'傳回 {len(retrieved_docs)} 筆資料')

傳回 6 筆資料


In [33]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [34]:
str_parser = StrOutputParser()
template = (
    "請根據以下內容加上自身判斷回答問題:\n"
    "{context}\n"
    "問題: {question}"
    )
prompt = ChatPromptTemplate.from_template(template)

In [35]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | str_parser
)

In [36]:
print(chain.invoke("汽車駕駛人若喝酒後，會使反應遲延，視力變差。請問是否正確"))

根據提供的文件內容來看，確實有提到酒後駕車會使反應遲延，視力變差，因此這句描述是正確的。喝酒後駕車將會危害自己和他人的生命安全，因此應嚴格禁止。


In [37]:
for chunk in chain.stream("汽車駕駛人若喝酒後，會使反應遲延，視力增加。請問是否正確"):
    print(chunk, end="", flush=True)

根據提供的文件內容可以得知，飲酒後會使視覺能力變差，運動反射神經遲鈍，肇事率增加，因此汽車駕駛人若喝酒後，會使反應遲延，視力增加這個說法是不正確的。飲酒後會對駕駛能力造成負面影響，不應該飲酒後駕駛汽車。

### 傳回關聯資料

In [ ]:
rag_chain_from_docs = (
    prompt
    | chat_model
    | StrOutputParser()
)

In [ ]:
from langchain_core.runnables import RunnableParallel

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
def chat(query):
    output = {}
    curr_key = None
    for chunk in rag_chain_with_source.stream(query):
        for key in chunk:
            if key not in output:
                output[key] = chunk[key]
            else:
                output[key] += chunk[key]
            if key != curr_key:
                print(f"\n\n{key}: {chunk[key]}", end="", flush=True)
            else:
                print(chunk[key], end="", flush=True)
            curr_key = key
chat("汽車駕駛人若喝酒後，會使反應遲延，視力變差。是否正確")

In [ ]:
chat("紅燈可以右轉。是否正確")

### RAG 集大成：建立檢索對話代理

In [ ]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever=retriever,
    name="retriever_by_car_regulations",
    description="搜尋並返回汽車法規是非題內容",
)
tools = [tool]

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ('system','你是一位善用工具的好助理, '
              '請自己判斷上下文來回答問題, 不要盲目地使用工具'),
    MessagesPlaceholder(variable_name="chat_history"),
    ('human','{input}'),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
from langchain.agents import (
    AgentExecutor, create_openai_tools_agent)

agent = create_openai_tools_agent(chat_model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
from langchain_community.chat_message_histories import (
    SQLChatMessageHistory)
from langchain_core.runnables.history import (
    RunnableWithMessageHistory)

memory = SQLChatMessageHistory(
        session_id="test_id",
        connection_string='sqlite:////content/drive/MyDrive/retriever.db'
    )

def window_messages(chain_input):
    if len(memory.messages) > 6:
        cur_messages = memory.messages
        memory.clear()
        for message in cur_messages[-6:]:
            memory.add_message(message)
    return

def add_history(agent_executor):
    agent_with_chat_history = RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: memory,
        input_messages_key="input",
        history_messages_key="chat_history",
    )
    memory_chain = (
        RunnablePassthrough.assign(messages=window_messages)
        | agent_with_chat_history
    )
    return memory_chain

In [ ]:
memory_chain = add_history(agent_executor)

In [ ]:
while True:
    msg = input("我說：")
    if not msg.strip():
        break
    for chunk in memory_chain.stream(
        {"input": msg},
        config={"configurable": {"session_id": "test_id"}}):
        if 'output' in chunk:
            print(f"AI 回覆：{chunk['output']}", end="", flush=True)
    print('\n')

## 7-4 總結文件的流程鏈

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/nvidia-nim/")
langchain_docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                               chunk_overlap=10)
langchain_splits = text_splitter.split_documents(langchain_docs)

### 不同的總結方式

stuff

In [ ]:
language_prompt = '使用繁體中文和台灣用詞'
prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}總結以下內容：\n\n{text}")]
).partial(language=language_prompt)

In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm=chat_model,
                             prompt=prompt,
                             chain_type="stuff")
print(chain.invoke(langchain_splits)['output_text'])

MapReduce

In [ ]:
reduce_prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}, 以下是文件內容：\n"
                "{text}\n"
                "將這些內容進行總結且保持核心內容")]
).partial(language=language_prompt)

map_prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}, 以下是一組文件串列：\n"
                "{text}\n"
                "根據此文件串列, 請作摘要並確保核心內容")]
).partial(language=language_prompt)

In [ ]:
chain = load_summarize_chain(llm=chat_model,
                             combine_prompt=reduce_prompt,
                             map_prompt=map_prompt,
                             chain_type="map_reduce")
print(chain.invoke(langchain_splits)['output_text'])

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}, 以下是文件的開頭內容：\n"
                "{text}\n"
                "將這些內容進行總結且保持核心內容")]
).partial(language=language_prompt)

refine_prompt = ChatPromptTemplate.from_messages(
    [("system", "{language}, 你的工作是撰寫綜合摘要\n"
                "這是目前的摘要成果：{existing_answer}\n"
                "藉由底下的額外內容"
                "（若需要的話）請再補強摘要內容：\n"
                "------------\n"
                "{text}\n"
                "------------\n"
                "如果這些額外內容沒有用，請返回原始摘要。")
    ]
).partial(language=language_prompt)

In [ ]:
chain = load_summarize_chain(chat_model,
                             question_prompt=prompt,
                             refine_prompt=refine_prompt,
                             chain_type="refine")
print(chain.invoke(langchain_splits)['output_text'])

## 7-5 其他的分割器

In [ ]:
json_example={
    "school": {
        "name": "精英中學",
        "address": {
            "street": "高科技大道200號",
            "city": "台北",
            "zip_code": "100"
        },
        "departments": [
            {
                "name": "數學系",
                "courses": [
                    {
                        "course_name": "高等數學",
                        "teacher": "李老師",
                        "students": ["張三", "李四", "王五"]
                    },
                    {
                        "course_name": "線性代數",
                        "teacher": "周老師",
                        "students": ["趙六", "孫七", "周八"]
                    }
                ]
            },
            {
                "name": "物理系",
                "courses": [
                    {
                        "course_name": "量子物理",
                        "teacher": "張老師",
                        "students": ["陳九", "鄭十"]
                    }
                ]
            }
        ]
    }
}

In [ ]:
from langchain.text_splitter import RecursiveJsonSplitter
splitter = RecursiveJsonSplitter(max_chunk_size=30)
docs = splitter.create_documents(texts=[json_example],
                                 convert_lists=True)
pprint(docs[:5])

### markdown

In [ ]:
md = '''
# 時間管理的藝術

時間管理是一項關鍵技能，可以幫助個人有效地利用時間，提高生產力和效率。
本文件旨在提供一些基本的時間管理技巧，幫助讀者更好地規劃和利用自己的時間。

## 為什麼時間管理如此重要？

在快節奏的現代生活中，時間成為了一種寶貴的資源。
良好的時間管理不僅可以幫助我們完成更多的工作，還可以提高生活質量，
給予我們更多時間去追求個人興趣和與家人、朋友相處的時光。

## 基本時間管理技巧

### 設定目標

- **確定優先順序**：了解哪些任務最重要，哪些可以稍後處理。
- **SMART目標**：設定具體（Specific）、可衡量（Measurable）、
可達成（Achievable）、相關（Relevant）、時間限定（Time-bound）的目標。

### 規劃你的時間

- **每日計劃**：每天制定一個實際可行的待辦事項清單。
- **時間塊劃分**：將一天分成幾個時間塊，每個時間塊分配特定的任務。

### 避免拖延

- **使用番茄工作法**：專注工作25分鐘，然後休息5分鐘。
- **設定獎勵**：完成任務後給自己一些小獎勵。

## 工具和應用

- **Google Calendar**：用於時間規劃和會議安排。
- **Trello**：一個項目管理工具，有助於跟蹤任務和進度。
- **Pomodoro Timer**：一個簡單的線上番茄鐘工具。

## 結語

有效的時間管理要求持之以恆的努力和自我反思。透過實踐上述技巧，
您將能夠更有效地利用您的時間，達成個人和專業目標，同時享有更豐富的個人生活。
'''

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=True)

md_header_splits = markdown_splitter.split_text(md)
pprint(md_header_splits)

In [ ]:
chunk_size = 50
chunk_overlap = 10

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(md_header_splits)
pprint(splits[:2])

### html

In [ ]:
from langchain_text_splitters import HTMLHeaderTextSplitter

In [ ]:
url = "https://zh.wikipedia.org/zh-tw/%E8%94%A1%E8%8B%B1%E6%96%87"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

html_header_splits = html_splitter.split_text_from_url(url)
pprint(html_header_splits[5:8])

In [ ]:
chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(html_header_splits)
pprint(splits[5:8])

### code

In [ ]:
from langchain_text_splitters import Language

In [ ]:
[e.value for e in Language]

In [ ]:
RecursiveCharacterTextSplitter.get_separators_for_language(
    Language.PYTHON)

In [ ]:
python_code = """
def hello_world():
    print("Hello, World!")

# 呼叫函式
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([python_code])
for doc in python_docs:
    print(doc.page_content)
    print('.'*10)

In [ ]:
csharp_code = """
using System;
class Program
{
    static void Main()
    {
        int age = 30; // 根據需要更改年齡值

        // 根據年齡對人進行分類，並輸出到控制台
        if (age < 18)
        {
            Console.WriteLine("年齡小於18：未成年");
        }
        else if (age >= 18 && age < 65)
        {
            Console.WriteLine("年齡在18至64之間：成年人");
        }
        else
        {
            Console.WriteLine("年齡65歲或以上：老年人");
        }
    }
}

"""

In [ ]:
RecursiveCharacterTextSplitter.get_separators_for_language(
    language=Language.CSHARP)

In [ ]:
csharp_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CSHARP, chunk_size=50, chunk_overlap=0
)
csharp_docs = csharp_splitter.create_documents([csharp_code])
for doc in csharp_docs:
    print(doc.page_content)
    print('.'*10)